# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
#Creating city dataframe
weather_df = pd.read_csv("../weatherpy/cities.csv")
weather_df.dropna()
weather_df.head()

"""
weather_data = pd.read_csv(weather_df)

weather_df = pd.DataFrame(weather_data)
weather_df.head()
"""

'\nweather_data = pd.read_csv(weather_df)\n\nweather_df = pd.DataFrame(weather_data)\nweather_df.head()\n'

In [20]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 10 columns):
City_ID       555 non-null int64
City          555 non-null object
Country       551 non-null object
Date          555 non-null int64
Latitude      555 non-null float64
Longitude     555 non-null float64
Cloudiness    555 non-null int64
Humidity      555 non-null int64
Max Temp      555 non-null float64
Wind Speed    555 non-null float64
dtypes: float64(4), int64(4), object(2)
memory usage: 43.5+ KB


In [21]:
weather_df.columns

Index(['City_ID', 'City', 'Country', 'Date', 'Latitude', 'Longitude',
       'Cloudiness', 'Humidity', 'Max Temp', 'Wind Speed'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [23]:
# Store 'Lat' and 'Lng' into  locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [24]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

temp_file=weather_df.loc[(weather_df['Max Temp']>=70)&(weather_df['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
96,96,Bubaque,GW,1579317847,11.28,-15.83,0,61,77.18,6.26
172,172,Moron,AR,1579317906,-34.65,-58.62,0,49,73.40,8.05
194,194,Harper,LR,1579317920,4.38,-7.72,0,94,76.91,1.68
273,273,Porto Nacional,BR,1579318105,-10.71,-48.42,0,94,78.80,4.70
313,313,Tura,IN,1579317853,25.52,90.22,0,55,72.32,1.79


In [26]:
ideal_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 96 to 548
Data columns (total 10 columns):
City_ID       15 non-null int64
City          15 non-null object
Country       15 non-null object
Date          15 non-null int64
Latitude      15 non-null float64
Longitude     15 non-null float64
Cloudiness    15 non-null int64
Humidity      15 non-null int64
Max Temp      15 non-null float64
Wind Speed    15 non-null float64
dtypes: float64(4), int64(4), object(2)
memory usage: 1.3+ KB


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 11.308134,
                    "lng": -15.921624
                },
                "viewport": {
                    "northeast": {
                        "lat": 11.30948382989272,
                        "lng": -15.92027417010727
                    },
                    "southwest": {
                        "lat": 11.30678417010728,
                        "lng": -15.92297382989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "8942c825208812f926f05de2f047a13554e1663d",
            "name": "Eco Lodge",
            "photos": [
                {
                    "height": 1836,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/105053556740367693992\">Maria Jo\u00e

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,96,Bubaque,GW,1579317847,11.28,-15.83,0,61,77.18,6.26,Eco Lodge
1,172,Moron,AR,1579317906,-34.65,-58.62,0,49,73.40,8.05,Hilton Buenos Aires
2,194,Harper,LR,1579317920,4.38,-7.72,0,94,76.91,1.68,Hotel Les Rochers
3,273,Porto Nacional,BR,1579318105,-10.71,-48.42,0,94,78.80,4.70,Vereda's Hotel
4,313,Tura,IN,1579317853,25.52,90.22,0,55,72.32,1.79,Hotel Polo Orchid
5,432,Dabhol,IN,1579318137,17.60,73.17,0,58,70.47,7.78,"Blue Ocean Resort and Spa , Ganpatipule"
6,439,Port Blair,IN,1579318141,11.67,92.75,0,88,78.80,2.24,"Taj Exotica Resort & Spa, Andamans"
7,442,Inirida,CO,1579318141,3.87,-67.92,0,92,73.98,1.99,Fuente del Guania Hotel de Lujo
8,443,Takaka,NZ,1579318141,-40.85,172.80,0,69,75.99,5.99,Awaroa Lodge
9,456,Bonthe,SL,1579318144,7.53,-12.51,0,88,77.07,0.96,Bonthe Holiday Village


In [28]:
hotel_df.head(15)

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,96,Bubaque,GW,1579317847,11.28,-15.83,0,61,77.18,6.26,Eco Lodge
1,172,Moron,AR,1579317906,-34.65,-58.62,0,49,73.40,8.05,Hilton Buenos Aires
2,194,Harper,LR,1579317920,4.38,-7.72,0,94,76.91,1.68,Hotel Les Rochers
3,273,Porto Nacional,BR,1579318105,-10.71,-48.42,0,94,78.80,4.70,Vereda's Hotel
4,313,Tura,IN,1579317853,25.52,90.22,0,55,72.32,1.79,Hotel Polo Orchid
5,432,Dabhol,IN,1579318137,17.60,73.17,0,58,70.47,7.78,"Blue Ocean Resort and Spa , Ganpatipule"
6,439,Port Blair,IN,1579318141,11.67,92.75,0,88,78.80,2.24,"Taj Exotica Resort & Spa, Andamans"
7,442,Inirida,CO,1579318141,3.87,-67.92,0,92,73.98,1.99,Fuente del Guania Hotel de Lujo
8,443,Takaka,NZ,1579318141,-40.85,172.80,0,69,75.99,5.99,Awaroa Lodge
9,456,Bonthe,SL,1579318144,7.53,-12.51,0,88,77.07,0.96,Bonthe Holiday Village


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [47]:
# Add marker layer ontop of heat map
fig=gmaps.figure()
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


plt.savefig("Heat_Map.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [45]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("Heat_Map.png")

# Display figure
fig

ValueError: weights must be of the same length as locations or None